In [1]:
import pandas as pd
import yfinance as yf
from tqdm import tqdm
import warnings
import logging
import sys
import os
import requests
from datetime import datetime

# Suppress all warnings
warnings.filterwarnings("ignore")

# Suppress logging from yfinance
logging.getLogger("yfinance").setLevel(logging.CRITICAL)

# Helper function to suppress print statements within a block of code
class SuppressPrint:
    def __enter__(self):
        self._original_stdout = sys.stdout
        self._original_stderr = sys.stderr
        sys.stdout = open(os.devnull, 'w')
        sys.stderr = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stderr.close()
        sys.stdout = self._original_stdout
        sys.stderr = self._original_stderr

def download_nasdaq_data(url, temp_file):
    """
    Download the latest NASDAQ data from the provided URL and save it to a temporary CSV file.
    """
    response = requests.get(url)
    if response.status_code == 200:
        with open(temp_file, 'w') as file:
            file.write(response.text)
        print(f"Downloaded NASDAQ data and saved to {temp_file}")
    else:
        print(f"Failed to download data. HTTP Status Code: {response.status_code}")
        sys.exit(1)

    # Load the downloaded data into a DataFrame
    data = pd.read_csv(temp_file, sep='|')
    data = data.dropna(subset=['Symbol'])  # Drop rows where the symbol is missing
    data.to_csv(temp_file, index=False)
    return data

def verify_symbols(data, nasdaq_file):
    """
    Verify tickers from the input DataFrame, add validation status, and save it as NASDAQ_Scripts.csv.
    """
    tickers = data['Symbol'].dropna().unique()

    validation_status = []
    validation_times = []

    with tqdm(total=len(tickers), desc="Verifying symbols", unit="ticker") as pbar:
        for ticker in tickers:
            start_time = datetime.now()
            with SuppressPrint():
                try:
                    stock = yf.Ticker(ticker)
                    hist = stock.history(period='max')
                    if not hist.empty:
                        validation_status.append((ticker, 'Valid'))
                    else:
                        validation_status.append((ticker, 'Invalid'))
                except Exception:
                    validation_status.append((ticker, 'Error'))
            end_time = datetime.now()
            validation_times.append((ticker, (end_time - start_time).total_seconds()))
            pbar.update(1)

    # Add validation status and time taken to the DataFrame
    validation_df = pd.DataFrame(validation_status, columns=['Symbol', 'Validation Status'])
    validation_time_df = pd.DataFrame(validation_times, columns=['Symbol', 'Validation Time (Seconds)'])
    updated_data = pd.merge(data, validation_df, on='Symbol', how='left')
    updated_data = pd.merge(updated_data, validation_time_df, on='Symbol', how='left')

    # Save the updated data to NASDAQ_Scripts.csv
    updated_data.to_csv(nasdaq_file, index=False)
    print(f"Updated {nasdaq_file} with validation status.")

    return updated_data

def fetch_data(updated_data, output_file, start_date=None, end_date=None):
    """
    Fetch historical data for valid tickers from the updated data and save it to the output_file.
    """
    valid_tickers = updated_data[updated_data['Validation Status'] == 'Valid']['Symbol'].tolist()

    all_data = pd.DataFrame()
    output_log = []

    with tqdm(total=len(valid_tickers), desc="Extracting data", unit="ticker") as pbar:
        for ticker in valid_tickers:
            start_time = datetime.now()
            with SuppressPrint():
                try:
                    stock = yf.Ticker(ticker)
                    hist = stock.history(start=start_date, end=end_date)  # Fetch historical data within date range
                    if not hist.empty:
                        last_date = hist.index.max()  # Get the latest date of available data
                        output_log.append((ticker, 'Valid', last_date))
                        hist.reset_index(inplace=True)  # Ensure the Date index is a column
                        hist['Ticker'] = ticker
                        all_data = pd.concat([all_data, hist])
                    else:
                        output_log.append((ticker, 'Invalid', pd.NaT))  # Use pd.NaT for missing datetime
                except Exception:
                    output_log.append((ticker, 'Error', pd.NaT))  # Use pd.NaT for errors
            end_time = datetime.now()
            output_log[-1] = (*output_log[-1], (end_time - start_time).total_seconds())
            pbar.update(1)

    if not all_data.empty:
        all_data.to_csv(output_file, index=False)
        print(f"Saved historical data to {output_file}")
    else:
        print("No data to save.")
    
    return output_log

def generate_output_log(output_log, output_log_file="output.csv"):
    """
    Generate the output log file with details for each ticker, including the Batch Run Timestamp.
    """
    batch_run_timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    output_df = pd.DataFrame(output_log, columns=['Symbol', 'Validation Status', 'Last Updated Date', 'Processing Time (Seconds)'])
    
    # Add the Batch Run Timestamp
    output_df['Batch Run Timestamp'] = batch_run_timestamp

    # Convert 'Last Updated Date' to datetime if it is not already, and format to 'YYYY-MM-DD'
    output_df['Last Updated Date'] = pd.to_datetime(output_df['Last Updated Date'], errors='coerce').dt.strftime('%Y-%m-%d')

    output_df.to_csv(output_log_file, index=False)
    print(f"Saved output log to {output_log_file}")

def log_batch_audit(batch_name, start_time, end_time, duration, status, comments, audit_log_file="batch_audit_trail.csv"):
    """
    Log the batch details including name, start time, end time, duration, status, and comments to a CSV file.
    """
    audit_data = pd.DataFrame([[batch_name, start_time, end_time, duration, status, comments]], 
                              columns=['Batch Name', 'Start Timestamp', 'End Timestamp', 'Duration (Minutes)', 'Status', 'Comments'])

    # If the file exists, append the data; otherwise, create a new file
    if os.path.exists(audit_log_file):
        audit_data.to_csv(audit_log_file, mode='a', header=False, index=False)
    else:
        audit_data.to_csv(audit_log_file, index=False)

def main(url, temp_file, nasdaq_file, output_file, output_log_file, audit_log_file, start_date=None, end_date=None):
    batch_name = "historical information refresh"
    start_time = datetime.now()  # Record the start time
    
    try:
        # Step 1: Download the latest NASDAQ data to a temporary file
        data = download_nasdaq_data(url, temp_file)

        # Step 2: Clone the temp file and validate symbols in NASDAQ_Scripts.csv
        updated_data = verify_symbols(data, nasdaq_file)

        # Step 3: Extract data and create all_stocks_data.csv
        output_log = fetch_data(updated_data, output_file, start_date, end_date)

        # Step 4: Generate output log with batch run timestamp
        generate_output_log(output_log, output_log_file)
        
        # Calculate the end time and duration
        end_time = datetime.now()  # Record the end time
        duration = (end_time - start_time).total_seconds() / 60.0  # Calculate duration in minutes

        # Prepare comments for the audit log
        valid_count = len([log for log in output_log if log[1] == 'Valid'])
        invalid_count = len([log for log in output_log if log[1] == 'Invalid'])
        error_count = len([log for log in output_log if log[1] == 'Error'])
        comments = (f"Batch completed. Valid tickers: {valid_count}, "
                    f"Invalid tickers: {invalid_count}, Errors: {error_count}.")
        if error_count > 0:
            error_tickers = [log[0] for log in output_log if log[1] == 'Error']
            comments += f" Errors identified in tickers: {', '.join(error_tickers)}."

        # Log the successful completion of the batch process
        log_batch_audit(batch_name, start_time, end_time, duration, "Completed", comments, audit_log_file)
        
        # Print the message after the audit log is written
        print(f"Batch audit logged to {audit_log_file}")
    
    except Exception as e:
        # Calculate the end time and duration in case of an error
        end_time = datetime.now()  # Record the end time
        duration = (end_time - start_time).total_seconds() / 60.0  # Calculate duration in minutes

        # Prepare error comments
        comments = f"Failed with error: {str(e)}"
        
        # Log the failure of the batch process
        log_batch_audit(batch_name, start_time, end_time, duration, "Failed", comments, audit_log_file)
        
        print(f"Error occurred: {str(e)}")
        sys.exit(1)

if __name__ == "__main__":
    # URLs and file paths
    nasdaq_url = 'https://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt'  # URL to fetch the latest NASDAQ data
    temp_file = 'temp_nasdaq_data.csv'  # Temporary file to save the downloaded NASDAQ data
    nasdaq_file = 'NASDAQ_Scripts.csv'  # File to save the cloned and validated NASDAQ data
    output_file = 'all_stocks_data.csv'  # File to save the historical data
    output_log_file = 'output.csv'  # File to save the output log with validation and last update info
    audit_log_file = 'batch_audit_trail.csv'  # File to save the batch audit trail

    # Configuration for historical data date range
    start_date = '1900-01-01'  # Start from a very early date
    end_date = datetime.now().strftime('%Y-%m-%d')  # End date is the current date
  
    print("Starting the process...")
    # Run the main process
    
    main(nasdaq_url, temp_file, nasdaq_file, output_file, output_log_file, audit_log_file, start_date, end_date)
    print("Process complete.")

Starting the process...
Downloaded NASDAQ data and saved to temp_nasdaq_data.csv


Verifying symbols: 100%|██████████████| 11045/11045 [51:55<00:00,  3.54ticker/s]


Updated NASDAQ_Scripts.csv with validation status.


Extracting data: 100%|██████████████| 10051/10051 [1:21:39<00:00,  2.05ticker/s]


Saved historical data to all_stocks_data.csv
Saved output log to output.csv
Batch audit logged to batch_audit_trail.csv
Process complete.
